In [19]:
## APPLIED DATA SCIENCE
## PROJECT 4 - GROUP 6
## A4 - WIP

In [20]:
#import libraries
import os,sys
import numpy as np
#import utils as ut
#import funcs_disp_mist as fdm

#from load_compas_data import *
sys.path.insert(0, '../../fair_classification/') # the code for fair classification is in this directory

#import for util class
from random import seed, shuffle
#import loss_funcs as lf # our implementation of loss funcs
from scipy.optimize import minimize # for loss func minimization
from multiprocessing import Pool, Process, Queue
from collections import defaultdict
from copy import deepcopy
import matplotlib.pyplot as plt # for plotting stuff

#import from func disp mis
import traceback
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
from cvxpy import *
import dccp
from dccp.problem import is_dccp
#import utils as ut

#import from loss funcs
import scipy.special
from collections import defaultdict
import traceback
from copy import deepcopy

#load compass data

import urllib.request, urllib.error, urllib.parse

import pandas as pd
#from collections import defaultdict
from sklearn import feature_extraction
from sklearn import preprocessing
#from random import seed, shuffle

sys.path.insert(0, '../../fair_classification/') # the code for fair classification is in this directory
#import utils as ut

In [21]:
#from load_compass_data

    sys.path.insert(0, '../../fair_classification/') # the code for fair classification is in this directory
    

    SEED = 1234
    seed(SEED)
    np.random.seed(SEED)

    """
        The adult dataset can be obtained from: https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
        The code will look for the data file in the present directory, if it is not found, it will download them from GitHub.
    """

    def check_data_file(fname):
        files = os.listdir(".") # get the current directory listing
        print("Looking for file '%s' in the current directory..." % fname)

        if fname not in files:
            print("'%s' not found! Downloading from GitHub..." % fname)
            addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
            response = urllib.request.urlopen(addr)
            data = response.read()
            fileOut = open(fname, "w")
            fileOut.write(data)
            fileOut.close()
            print("'%s' download and saved locally.." % fname)
        else:
            print("File found in current directory..")


    def load_compas_data():

        FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"] #features to be used for classification
        CONT_VARIABLES = ["priors_count"] # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
        CLASS_FEATURE = "two_year_recid" # the decision variable
        SENSITIVE_ATTRS = ["race"]


        COMPAS_INPUT_FILE = "compas-scores-two-years.csv"
        check_data_file(COMPAS_INPUT_FILE)

        # load the data and get some stats
        df = pd.read_csv(COMPAS_INPUT_FILE)
        df = df.dropna(subset=["days_b_screening_arrest"]) # dropping missing vals

        # convert to np array
        data = df.to_dict('list')
        for k in list(data.keys()):
            data[k] = np.array(data[k])


        """ Filtering the data """

        # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
        # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense. 
        idx = np.logical_and(data["days_b_screening_arrest"]<=30, data["days_b_screening_arrest"]>=-30)


        # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
        idx = np.logical_and(idx, data["is_recid"] != -1)

        # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
        idx = np.logical_and(idx, data["c_charge_degree"] != "O") # F: felony, M: misconduct

        # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
        idx = np.logical_and(idx, data["score_text"] != "NA")

        # we will only consider blacks and whites for this analysis
        idx = np.logical_and(idx, np.logical_or(data["race"] == "African-American", data["race"] == "Caucasian"))

        # select the examples that satisfy this criteria
        for k in list(data.keys()):
            data[k] = data[k][idx]



        """ Feature normalization and one hot encoding """

        # convert class label 0 to -1
        y = data[CLASS_FEATURE]
        y[y==0] = -1



        print("\nNumber of people recidivating within two years")
        print(pd.Series(y).value_counts())
        print("\n")


        X = np.array([]).reshape(len(y), 0) # empty array with num rows same as num examples, will hstack the features to it
        x_control = defaultdict(list)

        feature_names = []
        for attr in FEATURES_CLASSIFICATION:
            vals = data[attr]
            if attr in CONT_VARIABLES:
                vals = [float(v) for v in vals]
                vals = preprocessing.scale(vals) # 0 mean and 1 variance  
                vals = np.reshape(vals, (len(y), -1)) # convert from 1-d arr to a 2-d arr with one col

            else: # for binary categorical variables, the label binarizer uses just one var instead of two
                lb = preprocessing.LabelBinarizer()
                lb.fit(vals)
                vals = lb.transform(vals)

            # add to sensitive features dict
            if attr in SENSITIVE_ATTRS:
                x_control[attr] = vals


            # add to learnable features
            X = np.hstack((X, vals))

            if attr in CONT_VARIABLES: # continuous feature, just append the name
                feature_names.append(attr)
            else: # categorical features
                if vals.shape[1] == 1: # binary features that passed through lib binarizer
                    feature_names.append(attr)
                else:
                    for k in lb.classes_: # non-binary categorical features, need to add the names for each cat
                        feature_names.append(attr + "_" + str(k))


        # convert the sensitive feature to 1-d array
        x_control = dict(x_control)
        for k in list(x_control.keys()):
            assert(x_control[k].shape[1] == 1) # make sure that the sensitive feature is binary after one hot encoding
            x_control[k] = np.array(x_control[k]).flatten()

        # sys.exit(1)

        """permute the date randomly"""
        perm = list(range(0,X.shape[0]))
        shuffle(perm)
        X = X[perm]
        y = y[perm]
        for k in list(x_control.keys()):
            x_control[k] = x_control[k][perm]


        X = ut.add_intercept(X)

        feature_names = ["intercept"] + feature_names
        assert(len(feature_names) == X.shape[1])
        print("Features we will be using for classification are:", feature_names, "\n")


        return X, y, x_control


IndentationError: unexpected indent (2164442943.py, line 3)

In [22]:
#Utility class created from utils

class ut:
    SEED = 1122334455
    seed(SEED) # set the random seed so that the random permutations can be reproduced again
    np.random.seed(SEED)




    def train_model(x, y, x_control, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma=None):

        """

        Function that trains the model subject to various fairness constraints.
        If no constraints are given, then simply trains an unaltered classifier.
        Example usage in: "synthetic_data_demo/decision_boundary_demo.py"

        ----

        Inputs:

        X: (n) x (d+1) numpy array -- n = number of examples, d = number of features, one feature is the intercept
        y: 1-d numpy array (n entries)
        x_control: dictionary of the type {"s": [...]}, key "s" is the sensitive feature name, and the value is a 1-d list with n elements holding the sensitive feature values
        loss_function: the loss function that we want to optimize -- for now we have implementation of logistic loss, but other functions like hinge loss can also be added
        apply_fairness_constraints: optimize accuracy subject to fairness constraint (0/1 values)
        apply_accuracy_constraint: optimize fairness subject to accuracy constraint (0/1 values)
        sep_constraint: apply the fine grained accuracy constraint
            for details, see Section 3.3 of arxiv.org/abs/1507.05259v3
            For examples on how to apply these constraints, see "synthetic_data_demo/decision_boundary_demo.py"
        Note: both apply_fairness_constraints and apply_accuracy_constraint cannot be 1 at the same time
        sensitive_attrs: ["s1", "s2", ...], list of sensitive features for which to apply fairness constraint, all of these sensitive features should have a corresponding array in x_control
        sensitive_attrs_to_cov_thresh: the covariance threshold that the classifier should achieve (this is only needed when apply_fairness_constraints=1, not needed for the other two constraints)
        gamma: controls the loss in accuracy we are willing to incur when using apply_accuracy_constraint and sep_constraint

        ----

        Outputs:

        w: the learned weight vector for the classifier

        """


        assert((apply_accuracy_constraint == 1 and apply_fairness_constraints == 1) == False) # both constraints cannot be applied at the same time

        max_iter = 100000 # maximum number of iterations for the minimization algorithm

        if apply_fairness_constraints == 0:
            constraints = []
        else:
            constraints = get_constraint_list_cov(x, y, x_control, sensitive_attrs, sensitive_attrs_to_cov_thresh)      

        if apply_accuracy_constraint == 0: #its not the reverse problem, just train w with cross cov constraints

            f_args=(x, y)
            w = minimize(fun = loss_function,
                x0 = np.random.rand(x.shape[1],),
                args = f_args,
                method = 'SLSQP',
                options = {"maxiter":max_iter},
                constraints = constraints
                )

        else:

            # train on just the loss function
            w = minimize(fun = loss_function,
                x0 = np.random.rand(x.shape[1],),
                args = (x, y),
                method = 'SLSQP',
                options = {"maxiter":max_iter},
                constraints = []
                )

            old_w = deepcopy(w.x)


            def constraint_gamma_all(w, x, y,  initial_loss_arr):

                gamma_arr = np.ones_like(y) * gamma # set gamma for everyone
                new_loss = loss_function(w, x, y)
                old_loss = sum(initial_loss_arr)
                return ((1.0 + gamma) * old_loss) - new_loss

            def constraint_protected_people(w,x,y): # dont confuse the protected here with the sensitive feature protected/non-protected values -- protected here means that these points should not be misclassified to negative class
                return np.dot(w, x.T) # if this is positive, the constraint is satisfied
            def constraint_unprotected_people(w,ind,old_loss,x,y):

                new_loss = loss_function(w, np.array([x]), np.array(y))
                return ((1.0 + gamma) * old_loss) - new_loss

            constraints = []
            predicted_labels = np.sign(np.dot(w.x, x.T))
            unconstrained_loss_arr = loss_function(w.x, x, y, return_arr=True)

            if sep_constraint == True: # separate gemma for different people
                for i in range(0, len(predicted_labels)):
                    if predicted_labels[i] == 1.0 and x_control[sensitive_attrs[0]][i] == 1.0: # for now we are assuming just one sensitive attr for reverse constraint, later, extend the code to take into account multiple sensitive attrs
                        c = ({'type': 'ineq', 'fun': constraint_protected_people, 'args':(x[i], y[i])}) # this constraint makes sure that these people stay in the positive class even in the modified classifier             
                        constraints.append(c)
                    else:
                        c = ({'type': 'ineq', 'fun': constraint_unprotected_people, 'args':(i, unconstrained_loss_arr[i], x[i], y[i])})                
                        constraints.append(c)
            else: # same gamma for everyone
                c = ({'type': 'ineq', 'fun': constraint_gamma_all, 'args':(x,y,unconstrained_loss_arr)})
                constraints.append(c)

            def cross_cov_abs_optm_func(weight_vec, x_in, x_control_in_arr):
                cross_cov = (x_control_in_arr - np.mean(x_control_in_arr)) * np.dot(weight_vec, x_in.T)
                return float(abs(sum(cross_cov))) / float(x_in.shape[0])


            w = minimize(fun = cross_cov_abs_optm_func,
                x0 = old_w,
                args = (x, x_control[sensitive_attrs[0]]),
                method = 'SLSQP',
                options = {"maxiter":100000},
                constraints = constraints
                )

        try:
            assert(w.success == True)
        except:
            print("Optimization problem did not converge.. Check the solution returned by the optimizer.")
            print("Returned solution is:")
            print(w)



        return w.x


    def compute_cross_validation_error(x_all, y_all, x_control_all, num_folds, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh_arr, gamma=None):


        """
        Computes the cross validation error for the classifier subject to various fairness constraints
        This function is just a wrapper of "train_model(...)", all inputs (except for num_folds) are the same. See the specifications of train_model(...) for more info.

        Returns lists of train/test accuracy (with each list holding values for all folds), the fractions of various sensitive groups in positive class (for train and test sets), and covariance between sensitive feature and distance from decision boundary (again, for both train and test folds).
        """

        train_folds = []
        test_folds = []
        n_samples = len(y_all)
        train_fold_size = 0.7 # the rest of 0.3 is for testing

        # split the data into folds for cross-validation
        for i in range(0,num_folds):
            perm = list(range(0,n_samples)) # shuffle the data before creating each fold
            shuffle(perm)
            x_all_perm = x_all[perm]
            y_all_perm = y_all[perm]
            x_control_all_perm = {}
            for k in list(x_control_all.keys()):
                x_control_all_perm[k] = np.array(x_control_all[k])[perm]


            x_all_train, y_all_train, x_control_all_train, x_all_test, y_all_test, x_control_all_test = split_into_train_test(x_all_perm, y_all_perm, x_control_all_perm, train_fold_size)

            train_folds.append([x_all_train, y_all_train, x_control_all_train])
            test_folds.append([x_all_test, y_all_test, x_control_all_test])

        def train_test_single_fold(train_data, test_data, fold_num, output_folds, sensitive_attrs_to_cov_thresh):

            x_train, y_train, x_control_train = train_data
            x_test, y_test, x_control_test = test_data

            w = train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
            train_score, test_score, correct_answers_train, correct_answers_test = check_accuracy(w, x_train, y_train, x_test, y_test, None, None)

            distances_boundary_test = (np.dot(x_test, w)).tolist()
            all_class_labels_assigned_test = np.sign(distances_boundary_test)
            correlation_dict_test = get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
            cov_dict_test = print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)

            distances_boundary_train = (np.dot(x_train, w)).tolist()
            all_class_labels_assigned_train = np.sign(distances_boundary_train)
            correlation_dict_train = get_correlations(None, None, all_class_labels_assigned_train, x_control_train, sensitive_attrs)
            cov_dict_train = print_covariance_sensitive_attrs(None, x_train, distances_boundary_train, x_control_train, sensitive_attrs)

            output_folds.put([fold_num, test_score, train_score, correlation_dict_test, correlation_dict_train, cov_dict_test, cov_dict_train])


            return


        output_folds = Queue()
        processes = [Process(target=train_test_single_fold, args=(train_folds[x], test_folds[x], x, output_folds, sensitive_attrs_to_cov_thresh_arr[x])) for x in range(num_folds)]

        # Run processes
        for p in processes:
            p.start()


        # Get the reuslts
        results = [output_folds.get() for p in processes]
        for p in processes:
            p.join()


        test_acc_arr = []
        train_acc_arr = []
        correlation_dict_test_arr = []
        correlation_dict_train_arr = []
        cov_dict_test_arr = []
        cov_dict_train_arr = []

        results = sorted(results, key = lambda x : x[0]) # sort w.r.t fold num
        for res in results:
            fold_num, test_score, train_score, correlation_dict_test, correlation_dict_train, cov_dict_test, cov_dict_train = res

            test_acc_arr.append(test_score)
            train_acc_arr.append(train_score)
            correlation_dict_test_arr.append(correlation_dict_test)
            correlation_dict_train_arr.append(correlation_dict_train)
            cov_dict_test_arr.append(cov_dict_test)
            cov_dict_train_arr.append(cov_dict_train)


        return test_acc_arr, train_acc_arr, correlation_dict_test_arr, correlation_dict_train_arr, cov_dict_test_arr, cov_dict_train_arr



    def print_classifier_fairness_stats(acc_arr, correlation_dict_arr, cov_dict_arr, s_attr_name):

        correlation_dict = get_avg_correlation_dict(correlation_dict_arr)
        non_prot_pos = correlation_dict[s_attr_name][1][1]
        prot_pos = correlation_dict[s_attr_name][0][1]
        p_rule = (prot_pos / non_prot_pos) * 100.0

        print("Accuracy: %0.2f" % (np.mean(acc_arr)))
        print("Protected/non-protected in +ve class: %0.0f%% / %0.0f%%" % (prot_pos, non_prot_pos))
        print("P-rule achieved: %0.0f%%" % (p_rule))
        print("Covariance between sensitive feature and decision from distance boundary : %0.3f" % (np.mean([v[s_attr_name] for v in cov_dict_arr])))
        print()
        return p_rule

    def compute_p_rule(x_control, class_labels):

        """ Compute the p-rule based on Doctrine of disparate impact """

        non_prot_all = sum(x_control == 1.0) # non-protected group
        prot_all = sum(x_control == 0.0) # protected group
        non_prot_pos = sum(class_labels[x_control == 1.0] == 1.0) # non_protected in positive class
        prot_pos = sum(class_labels[x_control == 0.0] == 1.0) # protected in positive class
        frac_non_prot_pos = float(non_prot_pos) / float(non_prot_all)
        frac_prot_pos = float(prot_pos) / float(prot_all)
        p_rule = (frac_prot_pos / frac_non_prot_pos) * 100.0
        print()
        print("Total data points: %d" % (len(x_control)))
        print("# non-protected examples: %d" % (non_prot_all))
        print("# protected examples: %d" % (prot_all))
        print("Non-protected in positive class: %d (%0.0f%%)" % (non_prot_pos, non_prot_pos * 100.0 / non_prot_all))
        print("Protected in positive class: %d (%0.0f%%)" % (prot_pos, prot_pos * 100.0 / prot_all))
        print("P-rule is: %0.0f%%" % ( p_rule ))
        return p_rule




    def add_intercept(x):

        """ Add intercept to the data before linear classification """
        m,n = x.shape
        intercept = np.ones(m).reshape(m, 1) # the constant b
        return np.concatenate((intercept, x), axis = 1)

    def check_binary(arr):
        "give an array of values, see if the values are only 0 and 1"
        s = sorted(set(arr))
        if s[0] == 0 and s[1] == 1:
            return True
        else:
            return False

    def get_one_hot_encoding(in_arr):
        """
            input: 1-D arr with int vals -- if not int vals, will raise an error
            output: m (ndarray): one-hot encoded matrix
                    d (dict): also returns a dictionary original_val -> column in encoded matrix
        """

        for k in in_arr:
            if str(type(k)) != "<type 'numpy.float64'>" and type(k) != int and type(k) != np.int64:
                print(str(type(k)))
                print("************* ERROR: Input arr does not have integer types")
                return None

        in_arr = np.array(in_arr, dtype=int)
        assert(len(in_arr.shape)==1) # no column, means it was a 1-D arr
        attr_vals_uniq_sorted = sorted(list(set(in_arr)))
        num_uniq_vals = len(attr_vals_uniq_sorted)
        if (num_uniq_vals == 2) and (attr_vals_uniq_sorted[0] == 0 and attr_vals_uniq_sorted[1] == 1):
            return in_arr, None


        index_dict = {} # value to the column number
        for i in range(0,len(attr_vals_uniq_sorted)):
            val = attr_vals_uniq_sorted[i]
            index_dict[val] = i

        out_arr = []    
        for i in range(0,len(in_arr)):
            tup = np.zeros(num_uniq_vals)
            val = in_arr[i]
            ind = index_dict[val]
            tup[ind] = 1 # set that value of tuple to 1
            out_arr.append(tup)

        return np.array(out_arr), index_dict

    def check_accuracy(model, x_train, y_train, x_test, y_test, y_train_predicted, y_test_predicted):


        """
        returns the train/test accuracy of the model
        we either pass the model (w)
        else we pass y_predicted
        """
        if model is not None and y_test_predicted is not None:
            print("Either the model (w) or the predicted labels should be None")
            raise Exception("Either the model (w) or the predicted labels should be None")

        if model is not None:
            y_test_predicted = np.sign(np.dot(x_test, model))
            y_train_predicted = np.sign(np.dot(x_train, model))

        def get_accuracy(y, Y_predicted):
            correct_answers = (Y_predicted == y).astype(int) # will have 1 when the prediction and the actual label match
            accuracy = float(sum(correct_answers)) / float(len(correct_answers))
            return accuracy, sum(correct_answers)

        train_score, correct_answers_train = get_accuracy(y_train, y_train_predicted)
        test_score, correct_answers_test = get_accuracy(y_test, y_test_predicted)

        return train_score, test_score, correct_answers_train, correct_answers_test

    def test_sensitive_attr_constraint_cov(model, x_arr, y_arr_dist_boundary, x_control, thresh, verbose):


        """
        The covariance is computed b/w the sensitive attr val and the distance from the boundary
        If the model is None, we assume that the y_arr_dist_boundary contains the distace from the decision boundary
        If the model is not None, we just compute a dot product or model and x_arr
        for the case of SVM, we pass the distace from bounday becase the intercept in internalized for the class
        and we have compute the distance using the project function

        this function will return -1 if the constraint specified by thresh parameter is not satifsified
        otherwise it will reutrn +1
        if the return value is >=0, then the constraint is satisfied
        """




        assert(x_arr.shape[0] == x_control.shape[0])
        if len(x_control.shape) > 1: # make sure we just have one column in the array
            assert(x_control.shape[1] == 1)

        arr = []
        if model is None:
            arr = y_arr_dist_boundary # simply the output labels
        else:
            arr = np.dot(model, x_arr.T) # the product with the weight vector -- the sign of this is the output label

        arr = np.array(arr, dtype=np.float64)


        cov = np.dot(x_control - np.mean(x_control), arr ) / float(len(x_control))


        ans = thresh - abs(cov) # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
        # ans = thresh - cov # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
        if verbose is True:
            print("Covariance is", cov)
            print("Diff is:", ans)
            print()
        return ans

    def print_covariance_sensitive_attrs(model, x_arr, y_arr_dist_boundary, x_control, sensitive_attrs):


        """
        reutrns the covariance between sensitive features and distance from decision boundary
        """

        arr = []
        if model is None:
            arr = y_arr_dist_boundary # simplt the output labels
        else:
            arr = np.dot(model, x_arr.T) # the product with the weight vector -- the sign of this is the output label


        sensitive_attrs_to_cov_original = {}
        for attr in sensitive_attrs:

            attr_arr = x_control[attr]


            bin_attr = check_binary(attr_arr) # check if the attribute is binary (0/1), or has more than 2 vals
            if bin_attr == False: # if its a non-binary sensitive feature, then perform one-hot-encoding
                attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)

            thresh = 0

            if bin_attr:
                cov = thresh - test_sensitive_attr_constraint_cov(None, x_arr, arr, np.array(attr_arr), thresh, False)
                sensitive_attrs_to_cov_original[attr] = cov
            else: # sensitive feature has more than 2 categorical values            

                cov_arr = []
                sensitive_attrs_to_cov_original[attr] = {}
                for attr_val, ind in list(index_dict.items()):
                    t = attr_arr_transformed[:,ind]
                    cov = thresh - test_sensitive_attr_constraint_cov(None, x_arr, arr, t, thresh, False)
                    sensitive_attrs_to_cov_original[attr][attr_val] = cov
                    cov_arr.append(abs(cov))

                cov = max(cov_arr)

        return sensitive_attrs_to_cov_original


    def get_correlations(model, x_test, y_predicted, x_control_test, sensitive_attrs):


        """
        returns the fraction in positive class for sensitive feature values
        """

        if model is not None:
            y_predicted = np.sign(np.dot(x_test, model))

        y_predicted = np.array(y_predicted)

        out_dict = {}
        for attr in sensitive_attrs:

            attr_val = []
            for v in x_control_test[attr]: attr_val.append(v)
            assert(len(attr_val) == len(y_predicted))


            total_per_val = defaultdict(int)
            attr_to_class_labels_dict = defaultdict(lambda: defaultdict(int))

            for i in range(0, len(y_predicted)):
                val = attr_val[i]
                label = y_predicted[i]

                # val = attr_val_int_mapping_dict_reversed[val] # change values from intgers to actual names
                total_per_val[val] += 1
                attr_to_class_labels_dict[val][label] += 1

            class_labels = set(y_predicted.tolist())

            local_dict_1 = {}
            for k1,v1 in list(attr_to_class_labels_dict.items()):
                total_this_val = total_per_val[k1]

                local_dict_2 = {}
                for k2 in class_labels: # the order should be the same for printing
                    v2 = v1[k2]

                    f = float(v2) * 100.0 / float(total_this_val)


                    local_dict_2[k2] = f
                local_dict_1[k1] = local_dict_2
            out_dict[attr] = local_dict_1

        return out_dict



    def get_constraint_list_cov(x_train, y_train, x_control_train, sensitive_attrs, sensitive_attrs_to_cov_thresh):

        """
        get the list of constraints to be fed to the minimizer
        """

        constraints = []


        for attr in sensitive_attrs:


            attr_arr = x_control_train[attr]
            attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)

            if index_dict is None: # binary attribute
                thresh = sensitive_attrs_to_cov_thresh[attr]
                c = ({'type': 'ineq', 'fun': test_sensitive_attr_constraint_cov, 'args':(x_train, y_train, attr_arr_transformed,thresh, False)})
                constraints.append(c)
            else: # otherwise, its a categorical attribute, so we need to set the cov thresh for each value separately


                for attr_val, ind in list(index_dict.items()):
                    attr_name = attr_val                
                    thresh = sensitive_attrs_to_cov_thresh[attr][attr_name]

                    t = attr_arr_transformed[:,ind]
                    c = ({'type': 'ineq', 'fun': test_sensitive_attr_constraint_cov, 'args':(x_train, y_train, t ,thresh, False)})
                    constraints.append(c)


        return constraints



    def split_into_train_test(x_all, y_all, x_control_all, train_fold_size):

        split_point = int(round(float(x_all.shape[0]) * train_fold_size))
        x_all_train = x_all[:split_point]
        x_all_test = x_all[split_point:]
        y_all_train = y_all[:split_point]
        y_all_test = y_all[split_point:]
        x_control_all_train = {}
        x_control_all_test = {}
        for k in list(x_control_all.keys()):
            x_control_all_train[k] = x_control_all[k][:split_point]
            x_control_all_test[k] = x_control_all[k][split_point:]

        return x_all_train, y_all_train, x_control_all_train, x_all_test, y_all_test, x_control_all_test


    def get_avg_correlation_dict(correlation_dict_arr):
        # make the structure for the correlation dict
        correlation_dict_avg = {}
        # print correlation_dict_arr
        for k,v in list(correlation_dict_arr[0].items()):
            correlation_dict_avg[k] = {}
            for feature_val, feature_dict in list(v.items()):
                correlation_dict_avg[k][feature_val] = {}
                for class_label, frac_class in list(feature_dict.items()):
                    correlation_dict_avg[k][feature_val][class_label] = []

        # populate the correlation dict
        for correlation_dict in correlation_dict_arr:
            for k,v in list(correlation_dict.items()):
                for feature_val, feature_dict in list(v.items()):
                    for class_label, frac_class in list(feature_dict.items()):
                        correlation_dict_avg[k][feature_val][class_label].append(frac_class)

        # now take the averages
        for k,v in list(correlation_dict_avg.items()):
            for feature_val, feature_dict in list(v.items()):
                for class_label, frac_class_arr in list(feature_dict.items()):
                    correlation_dict_avg[k][feature_val][class_label] = np.mean(frac_class_arr)

        return correlation_dict_avg



    def plot_cov_thresh_vs_acc_pos_ratio(x_all, y_all, x_control_all, num_folds, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs):


        # very the covariance threshold using a range of decreasing multiplicative factors and see the tradeoffs between accuracy and fairness
        it = 0.05
        cov_range = np.arange(1.0, 0.0-it, -it).tolist()
        if apply_accuracy_constraint == True:
            if sep_constraint == False:
                it = 0.1
                cov_range = np.arange(0.0, 1.0 + it, it).tolist()
            if sep_constraint == True:
                cov_range =  [0,1,5,10,20,50,100,500,1000]


        positive_class_label = 1 # positive class is +1
        train_acc = []
        test_acc = []
        positive_per_category = defaultdict(list) # for each category (male / female), the frac of positive

        # first get the original values of covariance in the unconstrained classifier -- these original values are not needed for reverse constraint    
        test_acc_arr, train_acc_arr, correlation_dict_test_arr, correlation_dict_train_arr, cov_dict_test_arr, cov_dict_train_arr = compute_cross_validation_error(x_all, y_all, x_control_all, num_folds, loss_function, 0, apply_accuracy_constraint, sep_constraint, sensitive_attrs, [{} for i in range(0,num_folds)], 0)

        for c in cov_range:
            print("LOG: testing for multiplicative factor: %0.2f" % c)
            sensitive_attrs_to_cov_original_arr_multiplied = []
            for sensitive_attrs_to_cov_original in cov_dict_train_arr:
                sensitive_attrs_to_cov_thresh = deepcopy(sensitive_attrs_to_cov_original)
                for k in list(sensitive_attrs_to_cov_thresh.keys()):
                    v = sensitive_attrs_to_cov_thresh[k]
                    if type(v) == type({}):
                        for k1 in list(v.keys()):
                            v[k1] = v[k1] * c
                    else:
                        sensitive_attrs_to_cov_thresh[k] = v * c
                sensitive_attrs_to_cov_original_arr_multiplied.append(sensitive_attrs_to_cov_thresh)


            test_acc_arr, train_acc_arr, correlation_dict_test_arr, correlation_dict_train_arr, cov_dict_test_arr, cov_dict_train_arr  = compute_cross_validation_error(x_all, y_all, x_control_all, num_folds, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_original_arr_multiplied, c)
            test_acc.append(np.mean(test_acc_arr))


            correlation_dict_train = get_avg_correlation_dict(correlation_dict_train_arr)
            correlation_dict_test = get_avg_correlation_dict(correlation_dict_test_arr)

            # just plot the correlations for the first sensitive attr, the plotting can be extended for the other values, but as a proof of concept, we will jsut show for one
            s = sensitive_attrs[0]    

            for k,v in list(correlation_dict_test[s].items()):
                if v.get(positive_class_label) is None:
                    positive_per_category[k].append(0.0)
                else:
                    positive_per_category[k].append(v[positive_class_label])

        positive_per_category = dict(positive_per_category)

        p_rule_arr = (np.array(positive_per_category[0]) / np.array(positive_per_category[1])) * 100.0


        ax = plt.subplot(2,1,1)
        plt.plot(cov_range, positive_per_category[0], "-o" , color="green", label = "Protected")
        plt.plot(cov_range, positive_per_category[1], "-o", color="blue", label = "Non-protected")
        ax.set_xlim([min(cov_range), max(cov_range)])
        plt.xlabel('Multiplicative loss factor')
        plt.ylabel('Perc. in positive class')
        if apply_accuracy_constraint == False:
            plt.gca().invert_xaxis()
            plt.xlabel('Multiplicative covariance factor (c)')
        ax.legend()

        ax = plt.subplot(2,1,2)
        plt.scatter(p_rule_arr, test_acc, color="red")
        ax.set_xlim([min(p_rule_arr), max(max(p_rule_arr), 100)])
        plt.xlabel('P% rule')
        plt.ylabel('Accuracy')

        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
        plt.show()


    def get_line_coordinates(w, x1, x2):
        y1 = (-w[0] - (w[1] * x1)) / w[2]
        y2 = (-w[0] - (w[1] * x2)) / w[2]    
        return y1,y2

In [23]:
#class created from funcs_disp_mist

class fdm:
    SEED = 1122334455
    seed(SEED) # set the random seed so that the random permutations can be reproduced again
    np.random.seed(SEED)


    def train_model_disp_mist(x, y, x_control, loss_function, EPS, cons_params=None):

        # cons_type, sensitive_attrs_to_cov_thresh, take_initial_sol, gamma, tau, mu, EPS, cons_type
        """

        Function that trains the model subject to various fairness constraints.
        If no constraints are given, then simply trains an unaltered classifier.
        Example usage in: "disparate_mistreatment/synthetic_data_demo/decision_boundary_demo.py"

        ----

        Inputs:

        X: (n) x (d+1) numpy array -- n = number of examples, d = number of features, one feature is the intercept
        y: 1-d numpy array (n entries)
        x_control: dictionary of the type {"s": [...]}, key "s" is the sensitive feature name, and the value is a 1-d list with n elements holding the sensitive feature values
        loss_function: the loss function that we want to optimize -- for now we have implementation of logistic loss, but other functions like hinge loss can also be added
        EPS: stopping criteria for the convex solver. check the CVXPY documentation for details. default for CVXPY is 1e-6

        cons_params: is None when we do not want to apply any constraints
        otherwise: cons_params is a dict with keys as follows:
            - cons_type: 
                - 0 for all misclassifications 
                - 1 for FPR
                - 2 for FNR
                - 4 for both FPR and FNR
            - tau: DCCP parameter, controls how much weight to put on the constraints, if the constraints are not satisfied, then increase tau -- default is DCCP val 0.005
            - mu: DCCP parameter, controls the multiplicative factor by which the tau increases in each DCCP iteration -- default is the DCCP val 1.2
            - take_initial_sol: whether the starting point for DCCP should be the solution for the original (unconstrained) classifier -- default value is True
            - sensitive_attrs_to_cov_thresh: covariance threshold for each cons_type, eg, key 1 contains the FPR covariance
        ----

        Outputs:

        w: the learned weight vector for the classifier

        """

        max_iters = 100 # for the convex program
        max_iter_dccp = 50  # for the dccp algo


        num_points, num_features = x.shape
        w = Variable(num_features) # this is the weight vector

        # initialize a random value of w
        np.random.seed(112233)
        w.value = np.random.rand(x.shape[1])

        if cons_params is None: # just train a simple classifier, no fairness constraints
            constraints = []
        else:
            constraints = get_constraint_list_cov(x, y, x_control, cons_params["sensitive_attrs_to_cov_thresh"], cons_params["cons_type"], w)


        if loss_function == "logreg":
            # constructing the logistic loss problem
            loss = cvxpy.sum(  logistic( cvxpy.multiply(-y, x*w) )  ) / num_points # we are converting y to a diagonal matrix for consistent


        # sometimes, its a good idea to give a starting point to the constrained solver
        # this starting point for us is the solution to the unconstrained optimization problem
        # another option of starting point could be any feasible solution
        if cons_params is not None:
            if cons_params.get("take_initial_sol") is None: # true by default
                take_initial_sol = True
            elif cons_params["take_initial_sol"] == False:
                take_initial_sol = False

            if take_initial_sol == True: # get the initial solution
                p = Problem(Minimize(loss), [])
                p.solve()


        # construct the cvxpy problem
        prob = Problem(Minimize(loss), constraints)

        # print "\n\n"
        # print "Problem is DCP (disciplined convex program):", prob.is_dcp()
        # print "Problem is DCCP (disciplined convex-concave program):", is_dccp(prob)

        try:

            tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset
            if cons_params is not None: # in case we passed these parameters as a part of dccp constraints
                if cons_params.get("tau") is not None: tau = cons_params["tau"]
                if cons_params.get("mu") is not None: mu = cons_params["mu"]

            prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                solver=ECOS, verbose=False, 
                feastol=EPS, abstol=EPS, reltol=EPS,feastol_inacc=EPS, abstol_inacc=EPS, reltol_inacc=EPS,
                max_iters=max_iters, max_iter=max_iter_dccp)


            assert(prob.status == "Converged" or prob.status == "optimal")
            # print "Optimization done, problem status:", prob.status

        except:
            traceback.print_exc()
            sys.stdout.flush()
            sys.exit(1)


        # check that the fairness constraint is satisfied
        for f_c in constraints:
            #assert(f_c.value == True) # can comment this out if the solver fails too often, but make sure that the constraints are satisfied empirically. alternatively, consider increasing tau parameter
            pass


        w = np.array(w.value).flatten() # flatten converts it to a 1d array


        return w


    def get_clf_stats(w, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs):




        assert(len(sensitive_attrs) == 1) # ensure that we have just one sensitive attribute
        s_attr = sensitive_attrs[0] # for now, lets compute the accuracy for just one sensitive attr


        # compute distance from boundary
        distances_boundary_train = get_distance_boundary(w, x_train, x_control_train[s_attr])
        distances_boundary_test = get_distance_boundary(w, x_test, x_control_test[s_attr])

        # compute the class labels
        all_class_labels_assigned_train = np.sign(distances_boundary_train)
        all_class_labels_assigned_test = np.sign(distances_boundary_test)


        train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(None, x_train, y_train, x_test, y_test, all_class_labels_assigned_train, all_class_labels_assigned_test)


        cov_all_train = {}
        cov_all_test = {}
        for s_attr in sensitive_attrs:


            print_stats = False # we arent printing the stats for the train set to avoid clutter

            # uncomment these lines to print stats for the train fold
            # print "*** Train ***"
            # print "Accuracy: %0.3f" % (train_score)
            # print_stats = True
            s_attr_to_fp_fn_train = get_fpr_fnr_sensitive_features(y_train, all_class_labels_assigned_train, x_control_train, sensitive_attrs, print_stats)
            cov_all_train[s_attr] = get_sensitive_attr_constraint_fpr_fnr_cov(None, x_train, y_train, distances_boundary_train, x_control_train[s_attr]) 


            print("\n")
            print("Accuracy: %0.3f" % (test_score))
            print_stats = True # only print stats for the test fold
            s_attr_to_fp_fn_test = get_fpr_fnr_sensitive_features(y_test, all_class_labels_assigned_test, x_control_test, sensitive_attrs, print_stats)
            cov_all_test[s_attr] = get_sensitive_attr_constraint_fpr_fnr_cov(None, x_test, y_test, distances_boundary_test, x_control_test[s_attr]) 
            print("\n")

        return train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test

    def get_distance_boundary(w, x, s_attr_arr):

        """
            if we have boundaries per group, then use those separate boundaries for each sensitive group
            else, use the same weight vector for everything
        """

        distances_boundary = np.zeros(x.shape[0])
        if isinstance(w, dict): # if we have separate weight vectors per group
            for k in list(w.keys()): # for each w corresponding to each sensitive group
                d = np.dot(x, w[k])
                distances_boundary[s_attr_arr == k] = d[s_attr_arr == k] # set this distance only for people with this sensitive attr val
        else: # we just learn one w for everyone else
            distances_boundary = np.dot(x, w)
        return distances_boundary


    def get_constraint_list_cov(x_train, y_train, x_control_train, sensitive_attrs_to_cov_thresh, cons_type, w):

        """
        get the list of constraints to be fed to the minimizer

        cons_type == 0: means the whole combined misclassification constraint (without FNR or FPR)
        cons_type == 1: FPR constraint
        cons_type == 2: FNR constraint
        cons_type == 4: both FPR as well as FNR constraints

        sensitive_attrs_to_cov_thresh: is a dict like {s: {cov_type: val}}
        s is the sensitive attr
        cov_type is the covariance type. contains the covariance for all misclassifications, FPR and for FNR etc
        """

        constraints = []
        for attr in list(sensitive_attrs_to_cov_thresh.keys()):

            attr_arr = x_control_train[attr]
            attr_arr_transformed, index_dict = ut.get_one_hot_encoding(attr_arr)

            if index_dict is None: # binary attribute, in this case, the attr_arr_transformed is the same as the attr_arr

                s_val_to_total = {ct:{} for ct in [0,1,2]} # constrain type -> sens_attr_val -> total number
                s_val_to_avg = {ct:{} for ct in [0,1,2]}
                cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

                for v in set(attr_arr):
                    s_val_to_total[0][v] = np.sum(x_control_train[attr] == v)
                    s_val_to_total[1][v] = np.sum(np.logical_and(x_control_train[attr] == v, y_train == -1)) # FPR constraint so we only consider the ground truth negative dataset for computing the covariance
                    s_val_to_total[2][v] = np.sum(np.logical_and(x_control_train[attr] == v, y_train == +1))


                for ct in [0,1,2]:
                    s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1/N in our formulation, differs from one constraint type to another
                    s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0/N


                for v in set(attr_arr):

                    idx = x_control_train[attr] == v                


                    #################################################################
                    # #DCCP constraints
                    dist_bound_prod = cvxpy.multiply(y_train[idx], x_train[idx] * w) # y.f(x)

                    cons_sum_dict[0][v] = cvxpy.sum( cvxpy.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(x_train)) # avg misclassification distance from boundary
                    cons_sum_dict[1][v] = cvxpy.sum( cvxpy.minimum(0, cvxpy.multiply( (1 - y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[1][v] / np.sum(y_train == -1)) # avg false positive distance from boundary (only operates on the ground truth neg dataset)
                    cons_sum_dict[2][v] = cvxpy.sum( cvxpy.minimum(0, cvxpy.multiply( (1 + y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[2][v] / np.sum(y_train == +1)) # avg false negative distance from boundary
                    #################################################################


                if cons_type == 4:
                    cts = [1,2]
                elif cons_type in [0,1,2]:
                    cts = [cons_type]

                else:
                    raise Exception("Invalid constraint type")


                #################################################################
                #DCCP constraints
                for ct in cts:
                    thresh = abs(sensitive_attrs_to_cov_thresh[attr][ct][1] - sensitive_attrs_to_cov_thresh[attr][ct][0])
                    constraints.append( cons_sum_dict[ct][1] <= cons_sum_dict[ct][0]  + thresh )
                    constraints.append( cons_sum_dict[ct][1] >= cons_sum_dict[ct][0]  - thresh )

                #################################################################



            else: # otherwise, its a categorical attribute, so we need to set the cov thresh for each value separately
                # need to fill up this part
                raise Exception("Fill the constraint code for categorical sensitive features... Exiting...")
                sys.exit(1)


        return constraints


    def get_fpr_fnr_sensitive_features(y_true, y_pred, x_control, sensitive_attrs, verbose = False):



        # we will make some changes to x_control in this function, so make a copy in order to preserve the origianl referenced object
        x_control_internal = deepcopy(x_control)

        s_attr_to_fp_fn = {}

        for s in sensitive_attrs:
            s_attr_to_fp_fn[s] = {}
            s_attr_vals = x_control_internal[s]
            if verbose == True:
                print("||  s  || FPR. || FNR. ||")
            for s_val in sorted(list(set(s_attr_vals))):
                s_attr_to_fp_fn[s][s_val] = {}
                y_true_local = y_true[s_attr_vals==s_val]
                y_pred_local = y_pred[s_attr_vals==s_val]



                acc = float(np.sum(y_true_local==y_pred_local)) / len(y_true_local)

                fp = np.sum(np.logical_and(y_true_local == -1.0, y_pred_local == +1.0)) # something which is -ve but is misclassified as +ve
                fn = np.sum(np.logical_and(y_true_local == +1.0, y_pred_local == -1.0)) # something which is +ve but is misclassified as -ve
                tp = np.sum(np.logical_and(y_true_local == +1.0, y_pred_local == +1.0)) # something which is +ve AND is correctly classified as +ve
                tn = np.sum(np.logical_and(y_true_local == -1.0, y_pred_local == -1.0)) # something which is -ve AND is correctly classified as -ve

                all_neg = np.sum(y_true_local == -1.0)
                all_pos = np.sum(y_true_local == +1.0)

                fpr = float(fp) / float(fp + tn)
                fnr = float(fn) / float(fn + tp)
                tpr = float(tp) / float(tp + fn)
                tnr = float(tn) / float(tn + fp)


                s_attr_to_fp_fn[s][s_val]["fp"] = fp
                s_attr_to_fp_fn[s][s_val]["fn"] = fn
                s_attr_to_fp_fn[s][s_val]["fpr"] = fpr
                s_attr_to_fp_fn[s][s_val]["fnr"] = fnr

                s_attr_to_fp_fn[s][s_val]["acc"] = (tp + tn) / (tp + tn + fp + fn)
                if verbose == True:
                    if isinstance(s_val, float): # print the int value of the sensitive attr val
                        s_val = int(s_val)
                    print("||  %s  || %0.2f || %0.2f ||" % (s_val, fpr, fnr))


            return s_attr_to_fp_fn


    def get_sensitive_attr_constraint_fpr_fnr_cov(model, x_arr, y_arr_true, y_arr_dist_boundary, x_control_arr, verbose=False):


        """
        Here we compute the covariance between sensitive attr val and ONLY misclassification distances from boundary for False-positives
        (-N_1 / N) sum_0(min(0, y.f(x))) + (N_0 / N) sum_1(min(0, y.f(x))) for all misclassifications
        (-N_1 / N) sum_0(min(0, (1-y)/2 . y.f(x))) + (N_0 / N) sum_1(min(0,  (1-y)/2. y.f(x))) for FPR

        y_arr_true are the true class labels
        y_arr_dist_boundary are the predicted distances from the decision boundary

        If the model is None, we assume that the y_arr_dist_boundary contains the distace from the decision boundary
        If the model is not None, we just compute a dot product or model and x_arr
        for the case of SVM, we pass the distace from bounday becase the intercept in internalized for the class
        and we have compute the distance using the project function


        this function will return -1 if the constraint specified by thresh parameter is not satifsified
        otherwise it will reutrn +1
        if the return value is >=0, then the constraint is satisfied
        """


        assert(x_arr.shape[0] == x_control_arr.shape[0])
        if len(x_control_arr.shape) > 1: # make sure we just have one column in the array
            assert(x_control_arr.shape[1] == 1)
        if len(set(x_control_arr)) != 2: # non binary attr
            raise Exception("Non binary attr, fix to handle non bin attrs")


        arr = []
        if model is None:
            arr = y_arr_dist_boundary * y_arr_true # simply the output labels
        else:
            arr = np.dot(model, x_arr.T) * y_arr_true # the product with the weight vector -- the sign of this is the output label
        arr = np.array(arr)

        s_val_to_total = {ct:{} for ct in [0,1,2]}
        s_val_to_avg = {ct:{} for ct in [0,1,2]}
        cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

        for v in set(x_control_arr):
            s_val_to_total[0][v] = np.sum(x_control_arr == v)
            s_val_to_total[1][v] = np.sum(np.logical_and(x_control_arr == v, y_arr_true == -1))
            s_val_to_total[2][v] = np.sum(np.logical_and(x_control_arr == v, y_arr_true == +1))


        for ct in [0,1,2]:
            s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1 / N
            s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0 / N


        for v in set(x_control_arr):
            idx = x_control_arr == v
            dist_bound_prod = arr[idx]

            cons_sum_dict[0][v] = np.sum( np.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(x_arr))
            cons_sum_dict[1][v] = np.sum( np.minimum(0, ( (1 - y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[1][v] / np.sum(y_arr_true == -1))
            cons_sum_dict[2][v] = np.sum( np.minimum(0, ( (1 + y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[2][v] / np.sum(y_arr_true == +1))


        cons_type_to_name = {0:"ALL", 1:"FPR", 2:"FNR"}
        for cons_type in [0,1,2]:
            cov_type_name = cons_type_to_name[cons_type]    
            cov = cons_sum_dict[cons_type][1] - cons_sum_dict[cons_type][0]
            if verbose == True:
                print("Covariance for type '%s' is: %0.7f" %(cov_type_name, cov))

        return cons_sum_dict


    def plot_fairness_acc_tradeoff(x_all, y_all, x_control_all, loss_function, cons_type):


        # very the covariance threshold using a range of decreasing multiplicative factors and see the tradeoffs between accuracy and fairness
        it = 0.2
        mult_range = np.arange(1.0, 0.0-it, -it).tolist()




        positive_class_label = 1 # positive class is +1
        test_acc = []


        # first get the original values of covariance in the unconstrained classifier -- these original values are not needed for reverse constraint    
        test_acc_arr, train_acc_arr, correlation_dict_test_arr, correlation_dict_train_arr, cov_dict_test_arr, cov_dict_train_arr = compute_cross_validation_error(x_all, y_all, x_control_all, num_folds, loss_function, 0, apply_accuracy_constraint, sep_constraint, sensitive_attrs, [{} for i in range(0,num_folds)], 0)

        for c in cov_range:
            print("LOG: testing for multiplicative factor: %0.2f" % c)
            sensitive_attrs_to_cov_original_arr_multiplied = []
            for sensitive_attrs_to_cov_original in cov_dict_train_arr:
                sensitive_attrs_to_cov_thresh = deepcopy(sensitive_attrs_to_cov_original)
                for k in list(sensitive_attrs_to_cov_thresh.keys()):
                    v = sensitive_attrs_to_cov_thresh[k]
                    if type(v) == type({}):
                        for k1 in list(v.keys()):
                            v[k1] = v[k1] * c
                    else:
                        sensitive_attrs_to_cov_thresh[k] = v * c
                sensitive_attrs_to_cov_original_arr_multiplied.append(sensitive_attrs_to_cov_thresh)


            test_acc_arr, train_acc_arr, correlation_dict_test_arr, correlation_dict_train_arr, cov_dict_test_arr, cov_dict_train_arr  = compute_cross_validation_error(x_all, y_all, x_control_all, num_folds, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_original_arr_multiplied, c)
            test_acc.append(np.mean(test_acc_arr))


            correlation_dict_train = get_avg_correlation_dict(correlation_dict_train_arr)
            correlation_dict_test = get_avg_correlation_dict(correlation_dict_test_arr)

            # just plot the correlations for the first sensitive attr, the plotting can be extended for the other values, but as a proof of concept, we will jsut show for one
            s = sensitive_attrs[0]    

            for k,v in list(correlation_dict_test[s].items()):
                if v.get(positive_class_label) is None:
                    positive_per_category[k].append(0.0)
                else:
                    positive_per_category[k].append(v[positive_class_label])

        positive_per_category = dict(positive_per_category)

        p_rule_arr = (np.array(positive_per_category[0]) / np.array(positive_per_category[1])) * 100.0


        ax = plt.subplot(2,1,1)
        plt.plot(cov_range, positive_per_category[0], "-o" , color="green", label = "Protected")
        plt.plot(cov_range, positive_per_category[1], "-o", color="blue", label = "Non-protected")
        ax.set_xlim([min(cov_range), max(cov_range)])
        plt.xlabel('Multiplicative loss factor')
        plt.ylabel('Perc. in positive class')
        if apply_accuracy_constraint == False:
            plt.gca().invert_xaxis()
            plt.xlabel('Multiplicative covariance factor (c)')
        ax.legend()

        ax = plt.subplot(2,1,2)
        plt.scatter(p_rule_arr, test_acc, color="red")
        ax.set_xlim([min(p_rule_arr), max(max(p_rule_arr), 100)])
        plt.xlabel('P% rule')
        plt.ylabel('Accuracy')

        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
        plt.show()

In [25]:
#Class created from loss_funcs

class lf:
    def _hinge_loss(w, X, y):
        yz = y * np.dot(X,w) # y * (x.w)
        yz = np.maximum(np.zeros_like(yz), (1-yz)) # hinge function
        return sum(yz)

    def _logistic_loss(w, X, y, return_arr=None):
        """Computes the logistic loss.

        This function is used from scikit-learn source code

        Parameters
        ----------
        w : ndarray, shape (n_features,) or (n_features + 1,)
            Coefficient vector.

        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Training data.

        y : ndarray, shape (n_samples,)
            Array of labels.

        """


        yz = y * np.dot(X,w)
        # Logistic loss is the negative of the log of the logistic function.
        if return_arr == True:
            out = -(log_logistic(yz))
        else:
            out = -np.sum(log_logistic(yz))
        return out

    def _logistic_loss_l2_reg(w, X, y, lam=None):

        if lam is None:
            lam = 1.0

        yz = y * np.dot(X,w)
        # Logistic loss is the negative of the log of the logistic function.
        logistic_loss = -np.sum(log_logistic(yz))
        l2_reg = (float(lam)/2.0) * np.sum([elem*elem for elem in w])
        out = logistic_loss + l2_reg
        return out


    def log_logistic(X):

        """ This function is used from scikit-learn source code. Source link below """

        """Compute the log of the logistic function, ``log(1 / (1 + e ** -x))``.
        This implementation is numerically stable because it splits positive and
        negative values::
            -log(1 + exp(-x_i))     if x_i > 0
            x_i - log(1 + exp(x_i)) if x_i <= 0

        Parameters
        ----------
        X: array-like, shape (M, N)
            Argument to the logistic function

        Returns
        -------
        out: array, shape (M, N)
            Log of the logistic function evaluated at every point in x
        Notes
        -----
        Source code at:
        https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/extmath.py
        -----

        See the blog post describing this implementation:
        http://fa.bianp.net/blog/2013/numerical-optimizers-for-logistic-regression/
        """
        if X.ndim > 1: raise Exception("Array of samples cannot be more than 1-D!")
        out = np.empty_like(X) # same dimensions and data types

        idx = X>0
        out[idx] = -np.log(1.0 + np.exp(-X[idx]))
        out[~idx] = X[~idx] - np.log(1.0 + np.exp(X[~idx]))
        return out